In [1]:
import sys, os
if 'google.colab' in sys.modules and not os.path.exists('.setup_complete'):
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/setup_colab.sh -O- | bash
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week02_value_based/mdp.py
    !touch .setup_complete

# This code creates a virtual display to draw game images on.
# It will have no effect if your machine has a monitor.
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

# HW Part 1: Value iteration convergence

### Find an MDP for which value iteration takes long to converge  (1 pts)

When we ran value iteration on the small frozen lake problem, the last iteration where an action changed was iteration 6--i.e., value iteration computed the optimal policy at iteration 6. Are there any guarantees regarding how many iterations it'll take value iteration to compute the optimal policy? There are no such guarantees without additional assumptions--we can construct the MDP in such a way that the greedy policy will change after arbitrarily many iterations.

Your task: define an MDP with at most 3 states and 2 actions, such that when you run value iteration, the optimal action changes at iteration >= 50. Use discount=0.95. (However, note that the discount doesn't matter here--you can construct an appropriate MDP with any discount.)

Note: value function must change at least once after iteration >=50, not necessarily change on every iteration till >=50.

In [41]:
#found by brute-force a few cells below
p = [[0.0020946662477109167, 0.9954791609153069, 0.0024261728369821627], [0.4538413348620296, 0.004065787387917773, 0.5420928777500525], [0.522748024241617, 0.41834838748690134, 0.05890358827148161], [0.05590032913797978, 0.0017678255299463784, 0.9423318453320739], [0.12609838942288928, 0.17942788229668682, 0.6944737282804239], [0.9839084384646596, 0.015045046357288542, 0.0010465151780519042], [0.03483519001535749, 0.8119503721452164, 0.1532144378394262], [0.41088645989570227, 2.2527786594240957e-07, 0.5891133148264318], [0.7828775489485023, 0.023787120201481934, 0.19333533085001595], [0.137988056778625, 0.47657301770172766, 0.3854389255196474], [0.11761899526259705, 0.622869417924548, 0.25951158681285497], [0.6254538403214218, 0.01979950825775079, 0.3547466514208273], [0.02030840960043218, 0.7963690496829573, 0.1833225407166105], [0.9621013640962319, 0.0013187183567453036, 0.036579917547022694], [0.05177988885745235, 0.5488312270059246, 0.39938888413662305], [0.8476710608067988, 5.824102369925579e-06, 0.1523231150908313]]

transition_probs = {
      's0': {
          'a0': {'s0': p[0][0], 's1': p[0][1], 's2': p[0][2]},
          'a1': {'s0': p[1][0], 's1': p[1][1], 's2': p[1][2]}
      },
      's1': {
          'a0': {'s0': p[2][0], 's1': p[2][1], 's2': p[2][2]},
          'a1': {'s0': p[3][0], 's1': p[3][1], 's2': p[3][2]}
      },
      's2': {
          'a0': {'s0': p[4][0], 's1': p[4][1], 's2': p[4][2]},
          'a1': {'s0': p[5][0], 's1': p[5][1], 's2': p[5][2]}
      }
  }
rewards = {
      's0': {
        'a0': {'s0': p[6][0], 's1': p[6][1], 's2': p[6][2]},
        'a1': {'s0': p[7][0], 's1': p[7][1], 's2': p[7][2]},
        'a2': {'s0': p[8][0], 's1': p[8][1], 's2': p[8][2]}
        },
      's1': {
        'a0': {'s0': p[9][0], 's1': p[9][1], 's2': p[9][2]},
        'a1': {'s0': p[10][0], 's1': p[10][1], 's2': p[10][2]},
        'a2': {'s0': p[11][0], 's1': p[11][1], 's2': p[11][2]}
        },
      's2': {
        'a0': {'s0': p[12][0], 's1': p[12][1], 's2': p[12][2]},
        'a1': {'s0': p[13][0], 's1': p[13][1], 's2': p[13][2]},
        'a2': {'s0': p[14][0], 's1': p[14][1], 's2': p[14][2]}
        }
  }

from mdp import MDP
from mdp import FrozenLakeEnv
from numpy import random
import numpy as np 
mdp = MDP(transition_probs, rewards, initial_state=random.choice(tuple(transition_probs.keys())))
# Feel free to change the initial_state

In [50]:
#these functions were used in a cell below, but not declared in code
#I used templates for these functions taken from similar task from here:
#https://github.com/yandexdataschool/Practical_RL/blob/56df1f7612d5cb46b8343b7a95c6ae69900b0880/week02_value_based/seminar_vi.ipynb
#It works, I suppose

def get_action_value(mdp, state_values, state, action, gamma):
    """ Computes Q(s,a) as in formula above """
    next_states = mdp.get_next_states(state, action)
    arr = []
    for next_state, prob in next_states.items():
        arr.append(prob * (mdp.get_reward(state, action, next_state) + gamma*state_values[next_state]))
    return np.sum(arr)


def get_new_state_value(mdp, state_values, state, gamma):
    """ Computes next V(s) .Please do not change state_values in process. """
    if mdp.is_terminal(state):
        return 0
    arr = []
    for a in mdp.get_possible_actions(state):
        arr.append(get_action_value(mdp, state_values, state, a, gamma))
    return np.max(arr)


def get_optimal_action(mdp, state_values, state, gamma=0.9):
    """ Finds optimal action. """
    if mdp.is_terminal(state):
        return None
    map = {}
    for a in mdp.get_possible_actions(state):
        map[a] = get_action_value(mdp, state_values, state, a, gamma)
    return max(map, key=lambda key: map[key])

def value_iteration(mdp, state_values=None, gamma=0.9, num_iter=1000, min_difference=1e-5, verbose = False):
    """ performs num_iter value iteration steps starting from state_values. Same as before but in a function """
    zeros = {s: 0 for s in mdp.get_all_states()}
    state_values = state_values or zeros
    for i in range(num_iter):
        # Compute new state values using the functions you defined above. It must be a dict {state : new_V(state)}
        new_state_values = {}
        for s in state_values:
            new_state_values[s] = get_new_state_value(mdp, state_values, s, gamma)
        assert isinstance(new_state_values, dict)

        # Compute difference
        diff = max(abs(new_state_values[s] - state_values[s]) for s in mdp.get_all_states())
        if verbose:
            print("iter %4i   |   diff: %6.5f   |   V(start): %.3f " % (i, diff, new_state_values[mdp._initial_state]))

        state_values = new_state_values
        if diff < min_difference:
            break

    return state_values, i

In [51]:
gamma = 0.95

state_values = {s: 0 for s in mdp.get_all_states()}
policy = np.array([get_optimal_action(mdp, state_values, state, gamma)
                   for state in sorted(mdp.get_all_states())])

for i in range(100):
    state_values, _ = value_iteration(mdp, state_values, num_iter=1)

    new_policy = np.array([get_optimal_action(mdp, state_values, state, gamma)
                           for state in sorted(mdp.get_all_states())])

    n_changes = (policy != new_policy).sum()
    if (n_changes > 0):
        print("after iteration %i" % i)
        print("N actions changed = %i \n" % n_changes)
    policy = new_policy

# please ignore iter 0 at each step

after iteration 0
N actions changed = 2 

after iteration 2
N actions changed = 1 

after iteration 3
N actions changed = 1 

after iteration 5
N actions changed = 1 

after iteration 6
N actions changed = 1 

after iteration 8
N actions changed = 1 

after iteration 9
N actions changed = 1 

after iteration 11
N actions changed = 1 

after iteration 12
N actions changed = 1 

after iteration 14
N actions changed = 1 

after iteration 15
N actions changed = 1 

after iteration 17
N actions changed = 1 

after iteration 18
N actions changed = 1 

after iteration 20
N actions changed = 1 

after iteration 21
N actions changed = 1 

after iteration 23
N actions changed = 1 

after iteration 24
N actions changed = 1 

after iteration 26
N actions changed = 1 

after iteration 27
N actions changed = 1 

after iteration 29
N actions changed = 1 

after iteration 30
N actions changed = 1 

after iteration 32
N actions changed = 1 

after iteration 33
N actions changed = 1 

after iteration 34

In [40]:
#brute force to find the required MDP
p = [[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1],
     [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1],
     [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1]]
best_params = p
best_res = 0
max_iters = 100000
for _ in range(max_iters):
  for p1 in p:
    p1[0] = np.random.uniform(0, 1) ** 3
    p1[1] = np.random.uniform(0, 1) ** 4
    p1[2] = np.random.uniform(0, 1) ** 3
    s = p1[0] + p1[1] + p1[2]
    p1[0] /= s
    p1[1] /= s
    p1[2] /= s

  transition_probs = {
      's0': {
          'a0': {'s0': p[0][0], 's1': p[0][1], 's2': p[0][2]},
          'a1': {'s0': p[1][0], 's1': p[1][1], 's2': p[1][2]}
      },
      's1': {
          'a0': {'s0': p[2][0], 's1': p[2][1], 's2': p[2][2]},
          'a1': {'s0': p[3][0], 's1': p[3][1], 's2': p[3][2]}
      },
      's2': {
          'a0': {'s0': p[4][0], 's1': p[4][1], 's2': p[4][2]},
          'a1': {'s0': p[5][0], 's1': p[5][1], 's2': p[5][2]}
      }
  }
  rewards = {
      's0': {
        'a0': {'s0': p[6][0], 's1': p[6][1], 's2': p[6][2]},
        'a1': {'s0': p[7][0], 's1': p[7][1], 's2': p[7][2]},
        'a2': {'s0': p[8][0], 's1': p[8][1], 's2': p[8][2]}
        },
      's1': {
        'a0': {'s0': p[9][0], 's1': p[9][1], 's2': p[9][2]},
        'a1': {'s0': p[10][0], 's1': p[10][1], 's2': p[10][2]},
        'a2': {'s0': p[11][0], 's1': p[11][1], 's2': p[11][2]}
        },
      's2': {
        'a0': {'s0': p[12][0], 's1': p[12][1], 's2': p[12][2]},
        'a1': {'s0': p[13][0], 's1': p[13][1], 's2': p[13][2]},
        'a2': {'s0': p[14][0], 's1': p[14][1], 's2': p[14][2]}
        }
  }
  mdp = MDP(transition_probs, rewards, initial_state=random.choice(tuple(transition_probs.keys())))
  state_values = {s: 0 for s in mdp.get_all_states()}
  policy = np.array([get_optimal_action(mdp, state_values, state, gamma)
                    for state in sorted(mdp.get_all_states())])

  br = 0
  for i in range(100):
      state_values, _ = value_iteration(mdp, state_values, num_iter=1)

      new_policy = np.array([get_optimal_action(mdp, state_values, state, gamma)
                            for state in sorted(mdp.get_all_states())])

      n_changes = (policy != new_policy).sum()
      if (n_changes > 0):
          br = i
          #print("after iteration %i" % i)
          #print("N actions changed = %i \n" % n_changes)
      policy = new_policy
  if br > best_res:
    best_params = p
    best_res = br
    print("new best res ", br)
  if (best_res > 50):
    break
print("The best res ", best_res, " with params ", best_params)

new best res  4
new best res  11
new best res  13
new best res  19
new best res  22
new best res  28
new best res  30
new best res  54
The best res  54  with params  [[0.0020946662477109167, 0.9954791609153069, 0.0024261728369821627], [0.4538413348620296, 0.004065787387917773, 0.5420928777500525], [0.522748024241617, 0.41834838748690134, 0.05890358827148161], [0.05590032913797978, 0.0017678255299463784, 0.9423318453320739], [0.12609838942288928, 0.17942788229668682, 0.6944737282804239], [0.9839084384646596, 0.015045046357288542, 0.0010465151780519042], [0.03483519001535749, 0.8119503721452164, 0.1532144378394262], [0.41088645989570227, 2.2527786594240957e-07, 0.5891133148264318], [0.7828775489485023, 0.023787120201481934, 0.19333533085001595], [0.137988056778625, 0.47657301770172766, 0.3854389255196474], [0.11761899526259705, 0.622869417924548, 0.25951158681285497], [0.6254538403214218, 0.01979950825775079, 0.3547466514208273], [0.02030840960043218, 0.7963690496829573, 0.18332254071661

### Value iteration convervence proof (1 pts)
**Note:** Assume that $\mathcal{S}, \mathcal{A}$ are finite.

Update of value function in value iteration can be rewritten in a form of Bellman operator:

$$(TV)(s) = \max_{a \in \mathcal{A}}\mathbb{E}\left[ r_{k+1} + \gamma V(s_{k+1}) | s_t = s, a_t = a\right]$$

Value iteration algorithm with Bellman operator:

---
&nbsp;&nbsp; Initialize $V_0$

&nbsp;&nbsp; **for** $k = 0,1,2,...$ **do**

&nbsp;&nbsp;&nbsp;&nbsp; $V_{k+1} \leftarrow TV_k$

&nbsp;&nbsp;**end for**

---

In [lecture](https://docs.google.com/presentation/d/1lz2oIUTvd2MHWKEQSH8hquS66oe4MZ_eRvVViZs2uuE/edit#slide=id.g4fd6bae29e_2_4) we established contraction property of bellman operator:

$$
||TV - TU||_{\infty} \le \gamma ||V - U||_{\infty}
$$

For all $V, U$

Using contraction property of Bellman operator, Banach fixed-point theorem and Bellman equations prove that value function converges to $V^*$ in value iterateion$

Proof:
$V^*$ is a fixed point of operator $T$, then $TV^* = V^*$

Consider $||V_{k+1} - V^*||_{\infty}$ =  $||TV_{k} - TV^*||_{\infty}$ $\le$ $\gamma ||V_{k} - V^*||_{\infty}$ $\le$ $\gamma^{k+1} ||V_{0} - V^*||_{\infty}$

As $||U - V||_{\infty} \ge 0$, $||V_{0} - V^*||_{\infty}$ is finite, and $ 0 < \gamma < 1$, it means $\lim\limits_{k \to +\infty} ||V_{k} - V^*|| = 0$

Q.E.D.

### Asynchronious value iteration (2 pts)

Consider the following algorithm:

---

Initialize $V_0$

**for** $k = 0,1,2,...$ **do**

&nbsp;&nbsp;&nbsp;&nbsp; Select some state $s_k \in \mathcal{S}$    

&nbsp;&nbsp;&nbsp;&nbsp; $V(s_k) := (TV)(s_k)$

**end for**

---


Note that unlike common value iteration, here we update only a single state at a time.

**Homework.** Prove the following proposition:

If for all $s \in \mathcal{S}$, $s$ appears in the sequence $(s_0, s_1, ...)$ infinitely often, then $V$ converges to $V*$

Define $D_{i}^{k} = |V_{k}(s_i) - V^*(s_i)|$
Let's divide $\mathcal{S}$ into subsequences with the same states. All of them would be infinite.

$||V_{k+1}-V^*||_{\infty} \le ||V_{k+1}-V^*||_1 = \sum\limits_{i=1}^{|\mathcal{S}|} D_i^{k+1}$

Taking a limit on both sides:
$0 \le \lim\limits_{k \to +\infty} ||V_{k+1}-V^*||_{\infty} \le \lim\limits_{k \to +\infty}\sum\limits_{i=1}^{|\mathcal{S}|} D_i^{k+1}$ (1)

For each i $\lim\limits_{k \to +\infty} D_i^{k+1} = \lim\limits_{k \to +\infty} |V_{k+1}(s_i) - V^*(s_i)|$

Define $C_i(k) = \sum\limits_{j=1}^{k} \mathcal{I}(S[j] = s_i)$, $\mathcal{I}$ is an indicator function. $C_i(k) \to +\infty$ when $k \to +\infty$.

In this terms $\lim\limits_{k \to +\infty} D_i^{k+1} = \lim\limits_{k \to +\infty} |V_{k+1}(s_i) - V^*(s_i)| \le \lim\limits_{k \to +\infty} \gamma^{C_i(k)} |V_0(s_i) - V^*(s_i)| = 0$

So we can swap sum and limit operations
$\lim\limits_{k \to +\infty}\sum\limits_{i=1}^{|\mathcal{S}|} D_i^{k+1} = \sum\limits_{i=1}^{|\mathcal{S}|}\lim\limits_{k \to +\infty} D_i^{k+1}$

Put this into (1) and get $\lim\limits_{k \to +\infty} ||V_{k+1}-V^*||_{\infty} = 0$

Q.E.D.


# HW Part 2: Policy iteration

## Policy iteration implementateion (3 pts)

Let's implement exact policy iteration (PI), which has the following pseudocode:

---
Initialize $\pi_0$   `// random or fixed action`

For $n=0, 1, 2, \dots$
- Compute the state-value function $V^{\pi_{n}}$
- Using $V^{\pi_{n}}$, compute the state-action-value function $Q^{\pi_{n}}$
- Compute new policy $\pi_{n+1}(s) = \operatorname*{argmax}_a Q^{\pi_{n}}(s,a)$
---

Unlike VI, policy iteration has to maintain a policy - chosen actions from all states - and estimate $V^{\pi_{n}}$ based on this policy. It only changes policy once values converged.


Below are a few helpers that you may or may not use in your implementation.

In [73]:
transition_probs = {
    's0': {
        'a0': {'s0': 0.5, 's2': 0.5},
        'a1': {'s2': 1}
    },
    's1': {
        'a0': {'s0': 0.7, 's1': 0.1, 's2': 0.2},
        'a1': {'s1': 0.95, 's2': 0.05}
    },
    's2': {
        'a0': {'s0': 0.4, 's1': 0.6},
        'a1': {'s0': 0.3, 's1': 0.3, 's2': 0.4}
    }
}
rewards = {
    's1': {'a0': {'s0': +5}},
    's2': {'a1': {'s0': -1}}
}

from mdp import MDP
mdp = MDP(transition_probs, rewards, initial_state='s0')

Let's write a function called `compute_vpi` that computes the state-value function $V^{\pi}$ for an arbitrary policy $\pi$.

Unlike VI, this time you must find the exact solution, not just a single iteration.

Recall that $V^{\pi}$ satisfies the following linear equation:
$$V^{\pi}(s) = \sum_{s'} P(s,\pi(s),s')[ R(s,\pi(s),s') + \gamma V^{\pi}(s')]$$

You'll have to solve a linear system in your code. (Find an exact solution, e.g., with `np.linalg.solve`.)

In [74]:
def compute_vpi(mdp, policy, gamma):
    """
    Computes V^pi(s) FOR ALL STATES under given policy.
    :param policy: a dict of currently chosen actions {s : a}
    :returns: a dict {state : V^pi(state) for all states}
    """
    all_states = mdp.get_all_states()
    N = len(all_states)
    states_dict = dict(zip(all_states, np.arange(N)))
    system = np.diag(np.ones(N))
    A = np.zeros(N)
    for i, state in enumerate(all_states):
        if policy[state]:
            for next_state in mdp.get_next_states(state, policy[state]):
                prob = mdp.get_transition_prob(state, policy[state], next_state)
                reward = mdp.get_reward(state, policy[state], next_state)
                A[i] += prob * reward
                system[i][states_dict[next_state]] -= gamma * prob
            
    vpi = np.linalg.solve(system, A).tolist()
    return dict(zip(all_states, vpi))

In [75]:
test_policy = {s: np.random.choice(
    mdp.get_possible_actions(s)) for s in mdp.get_all_states()}
new_vpi = compute_vpi(mdp, test_policy, gamma)

print(new_vpi)

assert type(
    new_vpi) is dict, "compute_vpi must return a dict {state : V^pi(state) for all states}"

{'s0': -1.2155511811023618, 's1': -0.6545275590551177, 's2': -1.3435039370078736}


Once we've got new state values, it's time to update our policy.

In [76]:
def compute_qpi(mdp, vpi, gamma, state, action):
    arr = []
    for next_state in mdp.get_next_states(state, action):
        v = mdp.get_transition_prob(state, action, next_state)* (mdp.get_reward(state, action, next_state) + gamma * vpi[next_state])
        arr.append(v)
    return np.sum(arr)

def compute_new_policy(mdp, vpi, gamma):
    """
    Computes new policy as argmax of state values
    :param vpi: a dict {state : V^pi(state) for all states}
    :returns: a dict {state : optimal action for all states}
    """
    
    policy = {}
    for state in mdp.get_all_states():
        actions = mdp.get_possible_actions(state)
        if actions:
            actions_qpi = [compute_qpi(mdp, vpi, gamma, state, action) for action in actions]
            best_action_id = np.argmax(actions_qpi)
            policy[state] = actions[best_action_id]
        else:
            policy[state] = ''
            
    return policy

In [77]:
new_policy = compute_new_policy(mdp, new_vpi, gamma)

print(new_policy)

assert type(
    new_policy) is dict, "compute_new_policy must return a dict {state : optimal action for all states}"

{'s0': 'a0', 's1': 'a0', 's2': 'a0'}


__Main loop__

In [78]:
def random_policy(mdp):
    policy = {}
    for state in mdp.get_all_states():
        actions = mdp.get_possible_actions(state)
        if actions:
            policy[state] = actions[np.random.randint(0, len(actions))]
        else:
            policy[state] = ''
    return policy

def vpi_diff(vpi, new_vpi):
    return np.linalg.norm(np.array(list(new_vpi.values())) - np.array(list(vpi.values())), ord=np.inf)

def policy_iteration(mdp, policy=None, gamma=0.9, num_iter=1000, min_difference=1e-5):
    """ 
    Run the policy iteration loop for num_iter iterations or till difference between V(s) is below min_difference.
    If policy is not given, initialize it at random.
    """
    if policy is None:
        policy = random_policy(mdp)
    state_values = compute_vpi(mdp, policy, gamma)
    for i in range(num_iter):
        new_policy = compute_new_policy(mdp, state_values, gamma)
        new_state_values = compute_vpi(mdp, new_policy, gamma)
        diff = vpi_diff(state_values, new_state_values)
        policy = new_policy
        state_values = new_state_values
        if diff <= min_difference:
            break

    return state_values, policy, i

__Your PI Results__

In [82]:
def value_iteration_test(mdp, gamma=0.9, iters = 500):
    state_values, total_iters = value_iteration(mdp, gamma=gamma)

    reward_sums = []
    for _ in range(iters):
        s = mdp.reset()
        rewards = []
        for t in range(100):
            optimal_action = get_optimal_action(mdp, state_values, s, gamma)
            s, r, done, _ = mdp.step(optimal_action)
            rewards.append(r)
            if done:
                break
        reward_sums.append(np.sum(rewards))
    
    return np.mean(reward_sums), total_iters

def policy_iteration_test(mdp, gamma=0.9, iters = 500):
    vpi, policy, total_iters = policy_iteration(mdp, gamma=gamma)
    
    reward_sums = []
    for _ in range(iters):
        s = mdp.reset()
        rewards = []
        for t in range(100):
            optimal_action = policy[s]
            s, r, done, _ = mdp.step(optimal_action)
            rewards.append(r)
            if done:
                break
        reward_sums.append(np.sum(rewards))
    
    return np.mean(reward_sums), total_iters

def vi_pi_compare(mdp, gamma, iters, tries):
    
    vi_iters = []
    pi_iters = []
    
    vi_rewards = []
    pi_rewards = []
    
    for _ in range(tries):
        r, it = value_iteration_test(mdp, gamma, iters)
        vi_rewards.append(r)
        vi_iters.append(it)

        r, it = policy_iteration_test(mdp, gamma, iters)
        pi_rewards.append(r)
        pi_iters.append(it)

    print("Average VI reward ", np.mean(vi_rewards))
    print("Average iteration count:", np.mean(vi_iters))
    print("-----------------------")
    print("Average PI reward ", np.mean(pi_rewards))
    print("Average iteration count:", np.mean(pi_iters))

gamma = 0.9

mdp = FrozenLakeEnv(slip_chance=0.2)
print("Frozen lake")
vi_pi_compare(mdp, gamma, 500, 25)
print("-----------")

mdp = FrozenLakeEnv(slip_chance=0.2, map_name='8x8')
print("Large frozen lake")
vi_pi_compare(mdp, gamma, 500, 25)
print("-----------")

#policy iteration gives slitly higher reward and much less iteration needed to reach it

Frozen lake
Average VI reward  0.6496000000000001
Average iteration count: 21.0
-----------------------
Average PI reward  0.6593600000000001
Average iteration count: 4.28
-----------
Large frozen lake
Average VI reward  0.7409599999999998
Average iteration count: 33.0
-----------------------
Average PI reward  0.74472
Average iteration count: 7.36
-----------


## Policy iteration convergence (3 pts)

**Note:** Assume that $\mathcal{S}, \mathcal{A}$ are finite.

We can define another Bellman operator:

$$(T_{\pi}V)(s) = \mathbb{E}_{r, s'|s, a = \pi(s)}\left[r + \gamma V(s')\right]$$

And rewrite policy iteration algorithm in operator form:


---

Initialize $\pi_0$

**for** $k = 0,1,2,...$ **do**

&nbsp;&nbsp;&nbsp;&nbsp; Solve $V_k = T_{\pi_k}V_k$   

&nbsp;&nbsp;&nbsp;&nbsp; Select $\pi_{k+1}$ s.t. $T_{\pi_{k+1}}V_k = TV_k$

**end for**

---

To prove convergence of the algorithm we need to prove two properties: contraction an monotonicity.

#### Monotonicity (0.5 pts)

For all $V, U$ if $V(s) \le U(s)$   $\forall s \in \mathcal{S}$ then $(T_\pi V)(s) \le (T_\pi U)(s)$   $\forall s \in  \mathcal{S}$

Proof:


$T_\pi V(s) - T_\pi U(s) = \mathbb{E}_{r, s'|s, a = \pi(s)}\left[r + \gamma V(s')\right] - \mathbb{E}_{r, s'|s, a = \pi(s)}\left[r + \gamma U(s')\right] = $

$= \mathbb{E}_{r, s'|s, a = \pi(s)}\left[\gamma(V(s') - U(s'))\right]$ = {As $\mathcal{S}$ is finite, write expectation as a sum} = 

$ \sum\limits_{s'} \underbrace{P(r, s' | s, a = \pi(s))}_{\ge 0}\underbrace{\gamma}_{> 0}\underbrace{(V(s') - U(s'))}_{\le 0} \le 0$

Q.E.D.

#### Contraction (1 pts)

$$
||T_\pi V - T_\pi U||_{\infty} \le \gamma ||V - U||_{\infty}
$$

For all $V, U$

Proof:

$||T_\pi V - T_\pi U||_{\infty} = \max\limits_s \left|\sum\limits_{s'} P(r, s' | s, a = \pi(s))\gamma(V(s') - U(s'))\right| \le $

$\le \max\limits_s \sum\limits_{s'} \gamma P(r, s' | s, a = \pi(s)) |V(s') - U(s')| \le $

$\le \max\limits_s \sum\limits_{s'} \gamma\max\limits_{s'}|V(s') - U(s')|P(r, s' | s, a = \pi(s)) = $

$= \gamma||V - U||_{\infty}\max\limits_s \sum\limits_{s'} P(r, s' | s, a = \pi(s)) = $ {sum of probabilities for all states $s'$ equals to 1} = 

$= \gamma||V - U||_{\infty}$

Q.E.D.

#### Convergence (1.5 pts)

Prove that there exists iteration $k_0$ such that $\pi_k = \pi^*$ for all $k \ge k_0$

Proof:

We know about monotonicity of $V_k : V_k \le V_{k+1} \le V^*$ and using monotonivcity of $TV$ we get $T_{\pi_{k+1}} V_k \le T_{\pi_{k+1}}V_{k+1} = V_{k+1}$

Consider $|V_{k+1}-V^*| = V^* - V_{k+1} \le V^* - T_{\pi_{k+1}}V_k = T_{\pi_{k+1}}V^* - T_{\pi_{k+1}}V_k = |T_{\pi_{k+1}}V^* - T_{\pi_{k+1}}V_k|$

Then $||V_{k+1}-V^*||_{\infty} \le ||T_{\pi_{k+1}}V^* - T_{\pi_{k+1}}V_k||_{\infty} \le$ {contraction} $\le \gamma||V_k-V^*||_{\infty} \le \gamma^k||V_0-V^*||_{\infty}$

It means tnat $V_k$ monotonously converges to $V^*$. Hovewer, we have only a finite set of available policies $\Pi$ (as it's determined by actions and states, both finite) and all $V_k$ are produces by one of the policies from $\Pi$. Assume $\alpha = \min\limits_{\pi \in \Pi : ||V(\pi)-V^*||_{\infty} > 0}||V(\pi)-V^*||_{\infty}$.

Due to convergence $V_k : \exists k_0 : \forall k > k_0 \implies ||V_{k}-V^*||_{\infty} \le 0.5*\alpha$, but it means $\forall k > k_0 \implies ||V_{k}-V^*||_{\infty} = 0 $ due to definition of $\alpha$ and $\pi_k = \pi^*$ for all $k \ge k_0$

Q.E.D.